In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

### In this notebook, we will be filtering securities based on historical trading market value and relative spreads

As an example, we will download 1 month of historical stock data for listed companies in S&P500, aggregate the market value of each security, and pick the top 5% of securities.

In practice, quote data is required to calculate relative spread according to the formula:
    \begin{align*}
    \\
    &\mathbb{S} = \mathbb{E}\bigg[\cfrac{ask - bid}{mid}\bigg]
    && \mathbb{S} := \text{relative spread}\\
    &&& mid := \frac{ask+bid}{2}
    \end{align*}
    
Similarly, we would then pick the top 5% of securities with lowest relative spreads.

Without quote data, it is also possible to estimate relative spread based Farshid Abdi and Angelo Ranaldo formula:
    \begin{align*}
    \\
    &\large\tilde{\mathbb{S}} = \large\sqrt{\mathbb{E}[max\{4(c_t - \eta_t)(c_t - \eta_{t+1}), 0\}]}
    &    &c_t:= \text{log daily close price, time}\space t \\
    &&   &\eta_t := \text{midpoint of log daily high and log daily low, time} \space t  
    \end{align*}
    

References:
- F Abdi, A. Ronaldo. "A Simple Estimation of Bid-Ask Spreads from Daily Close, High, and Low Prices". The Review of Financial Studies, 30 (12): 4437-4480

### Assume today is 2020/02/01. we will use past 1 month (2020/01/01 - 2020/01/31) daily bar data to filter our universe.

In [2]:
listed_companies = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0].set_index('Symbol')

listed_companies.head()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Removing companies listed after 2020/01/01

# pd.to_datetime cannot be called directly due to inconsistent data structures
def try_mapping_to_datetime(date):
    try:
        return pd.Timestamp(date)
    except:
        return np.nan

listed_companies['Date first added'] = listed_companies['Date first added'].map(try_mapping_to_datetime)
listed_companies = listed_companies.dropna()
listed_companies = listed_companies[listed_companies['Date first added'] < pd.Timestamp('2020-01-01')]

listed_companies.head()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
full_universe = listed_companies.index.to_list()

historical_data = yf.download(tickers=full_universe,
                              start='2020-01-02',
                              end='2020-02-01',
                              show_errors=False)

# Remove securities that have been delisted.
historical_data = historical_data.dropna(axis=1)

historical_data.head()

[*********************100%***********************]  422 of 422 completed


Adj Close                                                          \
                    A        AAL         AAP       AAPL       ABBV        ABC   
Date                                                                            
2020-01-02  84.799675  28.982893  155.755661  73.988472  80.968918  82.331718   
2020-01-03  83.438141  27.548195  155.765411  73.269142  80.200363  81.296280   
2020-01-06  83.684792  27.219410  153.205490  73.852974  80.833290  82.486549   
2020-01-07  83.941315  27.119778  151.388107  73.505653  80.372162  81.896248   
2020-01-08  84.770073  27.737495  149.648941  74.688065  80.941788  82.689758   

                                                           ...   Volume  \
                  ABMD        ABT         ACN        ADBE  ...      XEL   
Date                                                       ...            
2020-01-02  168.809998  84.297447  204.304398  334.429993  ...  2915400   
2020-01-03  166.820007  83.269798  203.964157  331.809998  ...  2422500   
2020-01-06  179.039993  83.706055  202.632233  333.709991  ...  2217100   
2020-01-07  180.350006  83.240707  198.257416  333.390015  ...  1861100   
2020-01-08  178.690002  83.580025  198.646301  337.869995  ...  4828900   

                                                                           \
               XLNX       XOM     XRAY      XYL      YUM      ZBH    ZBRA   
Date                                                                        
2020-01-02  3234100  12456400  1556600   869500  1369900  1052400  387800   
2020-01-03  3521900  17386900   910000   795100  1145500   963500  305300   
2020-01-06  2832700  20081900   751000   817300  1454100   944100  322600   
2020-01-07  3050000  17387700   792600   662200  1388600   907300  236300   
2020-01-08  2172900  15137700  1001500  1083200  1521500   908800  689600   

                              
               ZION      ZTS  
Date                          
2020-01-02  1528700  1576700  
2020-01-03  1215800  1274000  
2020-01-06  1217500  2334100  
2020-01-07  1450900  1224500  
2020-01-08  2396000  1766700  

[5 rows x 2520 columns]

In [5]:
# Extracting raw data for processing
close = historical_data['Adj Close']
high = historical_data['High']
low = historical_data['Low']
volume = historical_data['Volume']

# creating indicators for market value and relative spread
market_value = close * volume
log_close = np.log(close)
log_mid = (np.log(high) + np.log(low)) / 2
log_mid_shifted = log_mid.shift(-1)  # eta_{t+1}

raw_indicator = 4 * (log_close - log_mid) * (log_close - log_mid_shifted)
raw_indicator = raw_indicator.applymap(lambda x: max(x, 0))
securities_and_market_values = market_value.sum(axis=0)
securities_and_relative_spread = raw_indicator.mean(axis=0)

# calculating quantile, top 5% highest market value, top 5% lowest relative spreads.
market_value_threshold = securities_and_market_values.quantile(.95)
relative_spread_threshold = securities_and_relative_spread.quantile(.5)

In [6]:
# filtering securities
filtered_universe = []

for security in full_universe:
    # remove securities that are delisted and have no historical data
    # or fail to meet market value / relative spread criterias
    if securities_and_market_values.get(security) is None or\
       securities_and_relative_spread.get(security) is None or\
       securities_and_market_values.get(security) < market_value_threshold or\
       securities_and_relative_spread.get(security) > relative_spread_threshold:
        
        continue
        
    filtered_universe.append(security)

### Filtered universe

In [7]:
listed_companies.loc[filtered_universe]

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
GOOGL,Alphabet (Class A),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,1652044,1998
GOOG,Alphabet (Class C),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2006-04-03,1652044,1998
AMZN,Amazon,reports,Consumer Discretionary,Internet & Direct Marketing Retail,"Seattle, Washington",2005-11-18,1018724,1994
AAPL,Apple,reports,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
BA,Boeing,reports,Industrials,Aerospace & Defense,"Chicago, Illinois",1957-03-04,12927,1916
MA,Mastercard,reports,Information Technology,Data Processing & Outsourced Services,"Harrison, New York",2008-07-18,1141391,1966
FB,Meta Platforms,reports,Communication Services,Interactive Media & Services,"Menlo Park, California",2013-12-23,1326801,2004
MU,Micron Technology,reports,Information Technology,Semiconductors,"Boise, Idaho",1994-09-27,723125,1978
MSFT,Microsoft,reports,Information Technology,Systems Software,"Redmond, Washington",1994-06-01,789019,1975
